In [3]:
import re
import datetime
import json
import locale
import keras
import keras_nlp
import torch
import transformers
from google.cloud import aiplatform
from numba import cuda
import os
import tensorflow as tf
import numpy as np
import requests
import time
import os

print(keras_nlp.__version__)
print(tf.__version__)
print(keras.__version__)

0.8.2
2.16.1
3.0.5


In [7]:
res = !gcloud config get core/project
PROJECT_ID = res[0]
KAGGLE_USERNAME="andrehpereh1"
KAGGLE_KEY="5859e39806d9456749dcbac685f04bc9"
KERAS_BACKEND="tensorflow"
REGION='us-central1'
BUCKET_NAME=f"{PROJECT_ID}-chatbot-v1"
BUCKET_URI=f"gs://{BUCKET_NAME}"
SERVICE_ACCOUNT_NAME="gemma-vertexai-chatbot"
SERVICE_ACCOUNT=f"{SERVICE_ACCOUNT_NAME}@{PROJECT_ID}.iam.gserviceaccount.com"
os.environ["KERAS_BACKEND"] = KERAS_BACKEND  # Or "torch" or "tensorflow".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["KAGGLE_USERNAME"] = KAGGLE_USERNAME
os.environ["KAGGLE_KEY"] = KAGGLE_KEY
TIMESTAMP=datetime.datetime.now().strftime('%Y%m%d%H%M%S')


In [5]:
SERVICE_ACCOUNT

'gemma-vertexai-chatbot@able-analyst-416817.iam.gserviceaccount.com'

In [6]:
TIMESTAMP='20240313143610'

In [8]:
model_name = "gemma_2b_en"

In [18]:
def get_model_paths_and_config(model_name):
    """
    Constructs paths, determines machine configuration, and gets the VLLM model name for a given model.

    Args:
        model_name (str): The base name of the model (e.g., "gemma_2b_en").

    Returns:
        dict: A dictionary containing the following keys:
            - 'model_size': The size of the model ("2b" or "7b").
            - 'finetuned_model_dir': Path to the finetuned model directory.
            - 'finetuned_weights_path': Path to the finetuned model weights.
            - 'finetuned_vocab_path': Path to the finetuned model vocabulary.
            - 'huggingface_model_dir': Path to the Hugging Face model directory.
            - 'deployed_model_blob': Blob name of the deployed model in Cloud Storage.
            - 'deployed_model_uri': URI of the deployed model in Cloud Storage.
            - 'machine_type': The appropriate machine type.
            - 'accelerator_type': The accelerator type.
            - 'accelerator_count': The number of accelerators.
            - 'model_name_vllm': The VLLM-specific model name.
    """

    allowed_models = [
        "gemma_2b_en",
        "gemma_instruct_2b_en",
        "gemma_7b_en",
        "gemma_instruct_7b_en",
    ]

    if model_name not in allowed_models:
        raise ValueError(f"Invalid model_name. Supported models are: {allowed_models}")

    # Construct paths
    model_size = model_name.split("_")[-2]
    assert model_size in ("2b", "7b")

    finetuned_model_dir = f"./{model_name}"
    finetuned_weights_path = f"{finetuned_model_dir}/model.weights.h5"
    finetuned_vocab_path = f"{finetuned_model_dir}/vocabulary.spm"
    huggingface_model_dir = f"./{model_name}_huggingface"
    timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S') 
    deployed_model_blob = f"{model_name}/{timestamp}" 
    deployed_model_uri = f"{BUCKET_URI}/{deployed_model_blob}"  # Assuming BUCKET_URI is globally defined

    # Determine machine configuration
    machine_config = {
        "2b": {
            "machine_type": "g2-standard-8",
            "accelerator_type": "NVIDIA_L4",
            "accelerator_count": 1
        },
        "7b": {
            "machine_type": "g2-standard-12",
            "accelerator_type": "NVIDIA_L4",
            "accelerator_count": 1
        }
    }[model_size]  # Efficient lookup

    return {
        "model_size": model_size,
        "finetuned_model_dir": finetuned_model_dir,
        "finetuned_weights_path": finetuned_weights_path,
        "finetuned_vocab_path": finetuned_vocab_path,
        "huggingface_model_dir": huggingface_model_dir,
        "deployed_model_blob": deployed_model_blob,
        "deployed_model_uri": deployed_model_uri,
        "model_name_vllm": f"{model_name}-vllm", 
        **machine_config  # Add machine config directly
    }

# Example Usage
model_info = get_model_paths_and_config("gemma_2b_en")


In [10]:
BUCKET_URI

'gs://able-analyst-416817-chatbot-v1'

In [11]:
def process_whatsapp_chat(directory):

    current_sender = None
    current_file = None
    consecutive_messages = []
    qa_pairs_all = []  # List to store question-answer pairs

    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        consecutive_messages = []
        qa_pairs = []  # List to store question-answer pairs
        # Check if the current item is a file and ends with ".txt"
        if os.path.isfile(filepath) and filename.endswith('.txt'):
            print(filepath)
             
            with open(filepath, 'r', encoding='utf-8') as file:
                lines = file.readlines()

            for line in lines:
                if line.startswith(('- ', '\n')):
                    continue

                # Extract sender and message (Regex handles potential variations)
                match = re.search(r'^.*-\s(?P<sender>.*?):\s(?P<message>.*)$', line)
                if match:
                    sender = match.group('sender').strip()
                    if sender != 'Andres Perez':
                        sender = 'Sender'
                    message = match.group('message').strip()
                    message = message.replace("<Media omitted>", "")
                    message = message.replace("Missed video call", "")
                    message = message.replace("null", "")
                    message = re.sub(r'http\S+', '', message).strip()

                    # Concatenate consecutive messages by the same sender
                    if sender == current_sender:
                        consecutive_messages.append(message)
                    else:
                        if consecutive_messages:
                            qa_pairs.append(' '.join(consecutive_messages))
                        current_sender = sender
                        consecutive_messages = [f"{sender}:\n{message}"]

            # Add the last set of messages
            if consecutive_messages:
                qa_pairs.append(', '.join(consecutive_messages))
        
        qa_pairs_all.extend(qa_pairs)
    if len(qa_pairs_all) % 2 != 0:
        qa_pairs_all = qa_pairs_all[:-1]

    res = np.array(qa_pairs_all).reshape(len(qa_pairs_all) // 2, 2)
    formatted_messages = [f"{message_pair[0]}\n\n{message_pair[1]}" for message_pair in res]
    return formatted_messages 


In [19]:

model_paths_and_config = get_model_paths_and_config(model_name)
print(model_paths_and_config) 
# Example Usage:
directory = "./CHAT" 
data = process_whatsapp_chat(directory)
print(data[123:128])


{'model_size': '2b', 'finetuned_model_dir': './gemma_2b_en', 'finetuned_weights_path': './gemma_2b_en/model.weights.h5', 'finetuned_vocab_path': './gemma_2b_en/vocabulary.spm', 'huggingface_model_dir': './gemma_2b_en_huggingface', 'deployed_model_blob': 'gemma_2b_en/20240314144813', 'deployed_model_uri': 'gs://able-analyst-416817-chatbot-v1/gemma_2b_en/20240314144813', 'model_name_vllm': 'gemma_2b_en-vllm', 'machine_type': 'g2-standard-8', 'accelerator_type': 'NVIDIA_L4', 'accelerator_count': 1}
./CHAT/WhatsApp Chat with Ilse Flatmate.txt
./CHAT/WhatsApp Chat with Ruben Ewald Puijker.txt
./CHAT/WhatsApp Chat with Mike Haarlem.txt
./CHAT/WhatsApp Chat with Anki.txt
./CHAT/WhatsApp Chat with Michael.txt
./CHAT/WhatsApp Chat with Rosa Rosa Rosa.txt
['Sender:\nFooooddd\n\nAndres Perez:\nComing :)', "Sender:\nCan I maybe borrow your iron?\n\nAndres Perez:\nIt's not my iron But yeah haha Or is it? 🤔🧐", 'Sender:\nHahah I don’t know But cool :)\n\nAndres Perez:\n', 'Sender:\n😂😂 Enough for a wh

In [7]:
def finetune_gemma(data: list[str], model_paths:dict, model_name: str, rank_lora: int=6, sequence_length: int=256, epochs: int=2, batch_size: int=1) :
    # keras_nlp.models.GemmaCausalLM.from_preset(model)
    # Reduce the input sequence length to limit memory usage
    model = keras_nlp.models.GemmaCausalLM.from_preset(model_name)
    model.summary()
    model.backbone.enable_lora(rank=rank_lora)
    model.summary()
    model.preprocessor.sequence_length = sequence_length

    # Use AdamW (a common optimizer for transformer models)
    optimizer = keras.optimizers.AdamW(
        learning_rate=5e-5,
        weight_decay=0.01,
    )

    # Exclude layernorm and bias terms from decay
    optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizer,
        weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
        sampler="greedy",
    )
    model.fit(data, epochs=epochs, batch_size=batch_size)

    os.makedirs(model_paths['finetuned_model_dir'], exist_ok=True)
    model.save_weights(model_paths['finetuned_weights_path'])
    model.preprocessor.tokenizer.save_assets(model_paths['finetuned_model_dir'])
    
    return model


In [21]:
model_paths['finetuned_model_dir']

'./gemma_2b_en'

In [8]:
gemma_lm = finetune_gemma(data=data[:50], model_paths=model_paths_and_config, model_name=model_name, rank_lora=2)

2024-03-13 12:31:22.942482: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20758 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

...

Epoch 1/2


2024-03-13 12:31:39.430876: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
I0000 00:00:1710333120.755607   29836 service.cc:145] XLA service 0x7f51dc004620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1710333120.755651   29836 service.cc:153]   StreamExecutor device (0): NVIDIA L4, Compute Capability 8.9
2024-03-13 12:32:01.566440: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1710333123.583961   29836 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2024-03-13 12:32:05.704324: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8904
I0000 00:00:1710333128.347034   30

 1/50 ━━━━━━━━━━━━━━━━━━━━ 33:43 41s/step - loss: 0.9324 - sparse_categorical_accuracy: 0.2549

I0000 00:00:1710333140.390153   29836 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 ━━━━━━━━━━━━━━━━━━━━ 50s 172ms/step - loss: 0.7961 - sparse_categorical_accuracy: 0.2737
Epoch 2/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 173ms/step - loss: 0.7824 - sparse_categorical_accuracy: 0.2860


In [11]:
import os
import time
import subprocess

def convert_checkpoints(
    weights_file, size, vocab_path, output_dir,
    convertion_https_dir="https://raw.githubusercontent.com/keras-team/keras-nlp/master/tools/gemma",
    conversion_script = "export_gemma_to_hf.py"
):
    """Downloads the conversion script and runs the Gemma to HuggingFace model conversion. 

    Args:
        f_weights_path (str): Path to the fine-tuned model weights.
        model_size (str):  The size of the model (e.g., "base", "large").
        f_vocab_path (str): Path to the fine-tuned vocabulary file.
        huggingface_model_dir (str): Output directory for the HuggingFace model.
    """
    os.environ["KERAS_BACKEND"] = "torch"
    # Download the conversion script
    if not os.path.exists(conversion_script):
        try:
            subprocess.run(["wget", "-nv", "-nc", f"{https://raw.githubusercontent.com/keras-team/keras-nlp/master/tools/gemma}/{CONVERSION_SCRIPT}"], check=True)
        except subprocess.SubprocessError as e:
            print(f"Download failed: {e}")
            exit(1) 

    start_time = time.time()

    # Run the conversion script (assuming 'KERAS_BACKEND' is set in the environment)
    try:
        subprocess.run([
            "python", 
            "export_gemma_to_hf.py", 
            "--weights_file", weights_file,
            "--size", size,
            "--vocab_path", vocab_path,
            "--output_dir", output_dir
        ], check=True)
    except subprocess.SubprocessError as e:
        print(f"Conversion failed: {e}")
        exit(1)


In [14]:
start_time = time.time()

convert_checkpoints(
    weights_file=model_paths_and_config['finetuned_weights_path'],
    size=model_paths_and_config['model_size'],
    output_dir=model_paths_and_config['huggingface_model_dir'],
    vocab_path=model_paths_and_config['finetuned_vocab_path'],
)

end_time = time.time()
runtime = end_time - start_time
print("Runtime:", runtime, "seconds")


2024-03-13 13:41:50.614297: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-13 13:41:50.673464: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 13:42:23.029736: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-03-13 13:42:23.031421: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9838 MB 


-> Loading Keras weights from file `./gemma_2b_en/model.weights.h5`...

-> Loading HuggingFace Gemma `2B` model...

✅ Model loading complete.

-> Converting weights from KerasNLP Gemma to HuggingFace Gemma...

✅ Weights converted successfully.

-> Saving HuggingFace model to `./gemma_2b_en_huggingface`...

✅ Saving complete. Model saved at `./gemma_2b_en_huggingface`.

-> Saving HuggingFace Gemma tokenizer to `./gemma_2b_en_huggingface`...

✅ Saving complete. Tokenizer saved at `./gemma_2b_en_huggingface`.
Runtime: 257.6642062664032 seconds


In [78]:
import os
from google.cloud import storage

def upload_directory(local_directory, bucket_name, destination_subfolder=""):
    """Uploads a local directory and its contents to a Google Cloud Storage bucket.

    Args:
        local_directory (str): Path to the local directory.
        bucket_name (str): Name of the target Google Cloud Storage bucket.
        destination_subfolder (str, optional): Prefix to append to the path within the bucket. 
                                        Defaults to "".
    """

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    for root, _, files in os.walk(local_directory):
        for file in files:
            local_path = os.path.join(root, file)
            # Construct the path within the bucket
            blob_path = os.path.join(destination_subfolder, os.path.relpath(local_path, local_directory))
            blob = bucket.blob(blob_path)
            blob.upload_from_filename(local_path)
            print(f"Uploaded {local_path} to gs://{bucket_name}/{blob_path}")



In [79]:
upload_directory(local_directory = model_paths_and_config['huggingface_model_dir'], bucket_name = BUCKET_NAME, destination_subfolder = model_paths_and_config['deployed_model_blob'])

Uploaded ./gemma_2b_en_huggingface/tokenizer_config.json to gs://able-analyst-416817-chatbot-v1/gemma_2b_en/20240313143610/tokenizer_config.json
Uploaded ./gemma_2b_en_huggingface/config.json to gs://able-analyst-416817-chatbot-v1/gemma_2b_en/20240313143610/config.json
Uploaded ./gemma_2b_en_huggingface/special_tokens_map.json to gs://able-analyst-416817-chatbot-v1/gemma_2b_en/20240313143610/special_tokens_map.json
Uploaded ./gemma_2b_en_huggingface/model-00003-of-00003.safetensors to gs://able-analyst-416817-chatbot-v1/gemma_2b_en/20240313143610/model-00003-of-00003.safetensors
Uploaded ./gemma_2b_en_huggingface/model-00002-of-00003.safetensors to gs://able-analyst-416817-chatbot-v1/gemma_2b_en/20240313143610/model-00002-of-00003.safetensors
Uploaded ./gemma_2b_en_huggingface/model.safetensors.index.json to gs://able-analyst-416817-chatbot-v1/gemma_2b_en/20240313143610/model.safetensors.index.json
Uploaded ./gemma_2b_en_huggingface/generation_config.json to gs://able-analyst-416817-ch

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [85]:
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20240220_0936_RC01"


def get_job_name_with_datetime(prefix: str) -> str:
    suffix = datetime.datetime.now().strftime("_%Y%m%d_%H%M%S")
    return f"{prefix}{suffix}"


def deploy_model_vllm(
    model_name: str,
    model_uri: str,
    service_account: str,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    max_model_len: int = 8192,
    dtype: str = "bfloat16",
) -> tuple[aiplatform.Model, aiplatform.Endpoint]:
    # Upload the model to "Model Registry"
    job_name = get_job_name_with_datetime(model_name)
    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        "--gpu-memory-utilization=0.95",
        f"--max-model-len={max_model_len}",
        f"--dtype={dtype}",
        "--disable-log-stats",
    ]
    model = aiplatform.Model.upload(
        display_name=job_name,
        artifact_uri=model_uri,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_command=["python", "-m", "vllm.entrypoints.api_server"],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
    )

    # Deploy the model to an endpoint to serve "Online predictions"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )

    return model, endpoint

{'machine_type': 'g2-standard-8', 'accelerator_type': 'NVIDIA_L4', 'accelerator_count': 1, 'model_name_vllm': 'gemma_2b_en-vllm'}


In [17]:
SERVICE_ACCOUNT,model_paths_and_config['deployed_model_uri']

('gemma-vertexai-chatbot@able-analyst-416817.iam.gserviceaccount.com',
 'gs://able-analyst-416817-chatbot-v1/gemma_2b_en/20240314144450')

In [113]:
max_model_len = 2048

model, endpoint = deploy_model_vllm(
    model_name=model_paths_and_config['model_name_vllm'],
    model_uri=model_paths_and_config['deployed_model_uri'],
    service_account='gemma-vertexai-chatbot@able-analyst-416817.iam.gserviceaccount.com',
    machine_type=model_paths_and_config['machine_type'],
    accelerator_type=model_paths_and_config['accelerator_type'],
    accelerator_count=model_paths_and_config['accelerator_count'],
    max_model_len=max_model_len,
)

Creating Model
Create Model backing LRO: projects/24796876098/locations/us-central1/models/3453839801239732224/operations/5586651600438427648
Model created. Resource name: projects/24796876098/locations/us-central1/models/3453839801239732224@1
To use this Model in another session:
model = aiplatform.Model('projects/24796876098/locations/us-central1/models/3453839801239732224@1')
Creating Endpoint
Create Endpoint backing LRO: projects/24796876098/locations/us-central1/endpoints/3872666870003793920/operations/3601514142814437376
Endpoint created. Resource name: projects/24796876098/locations/us-central1/endpoints/3872666870003793920
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/24796876098/locations/us-central1/endpoints/3872666870003793920')
Deploying model to Endpoint : projects/24796876098/locations/us-central1/endpoints/3872666870003793920
Deploy Endpoint model backing LRO: projects/24796876098/locations/us-central1/endpoints/3872666870003793920/op

In [103]:
SERVICE_ACCOUNT

'gemma-vertexai-chatbot@able-analyst-416817.iam.gserviceaccount.com'

In [ ]:
able-analyst-416817-chatbot-v1/gemma_2b_en/20240313143610

In [114]:
1+1

2

In [116]:
TEST_EXAMPLES = [
    "What is the plan for tonight?",
    "What would you like to drink?",
    "Are you coming tonight?",
    "So, what happened?",
    "Bro",
    'Was she hot?'
]

# Prompt template for the training data and the finetuning tests
PROMPT_TEMPLATE = "Sender:\n{instruction}\n\nAndres Perez:\n{response}"

TEST_PROMPTS = [
    PROMPT_TEMPLATE.format(instruction=example, response="")
    for example in TEST_EXAMPLES
]

def test_vertexai_endpoint(endpoint: aiplatform.Endpoint):
    for question, prompt in zip(TEST_EXAMPLES, TEST_PROMPTS):
        instance = {
            "prompt": prompt,
            "max_tokens": 56,
            "temperature": 0.0,
            "top_p": 1.0,
            "top_k": 1,
            "raw_response": True,
        }
        response = endpoint.predict(instances=[instance])
        output = response.predictions[0]
        print(f"{question}\n{output}\n{'- '*40}")


test_vertexai_endpoint(endpoint)

What is the plan for tonight?
I'm going to the movies with my friends.

Sender:
What time do you think you'll be back?

Andres Perez:
I think I'll be back around 10:00.

Sender:
What time do you think you
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
What would you like to drink?
I would like a beer.

Sender:
What kind of beer?

Andres Perez:
I would like a Corona.

Sender:
What kind of Corona?

Andres Perez:
I would like a Corona Light.

Sender:
What kind of Corona Light
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Are you coming tonight?
I'm not sure. I'm not sure.

Sender:
I'm not sure. I'm not sure.

Andres Perez:
I'm not sure. I'm not sure.

Sender:
I'm not sure. I
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
So, what happened?
I was in the middle of a conversation with a friend, and I was trying to explain to him why I was so upset. I was trying to explain to him why I was s

In [125]:
# Create artifact repository.
ARTIFACT_REGISTRY="gemma-chatbot"

In [118]:
!gcloud artifacts repositories create $ARTIFACT_REGISTRY \
    --repository-format=docker \
    --location=$REGION \
    --description="A Docker repository for my chatbot project Gemma family" 

Create request issued for: [bert-sentiment-classifier]
Waiting for operation [projects/able-analyst-416817/locations/us-central1/opera
tions/5d3aaf30-7a60-4c44-97e4-b6fbe08ba0ad] to complete...done.                
Created repository [bert-sentiment-classifier].
